In [58]:
import pandas as pd
import cleaningtools as ct 
import file_config as fconfig
import math


In [59]:
fconfig.assay_fname

'drill assays master.csv'

In [66]:
file=fconfig.hyp_file
data=pd.read_csv(file)


In [70]:
data[data.Sample_ID.isna()].index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
            68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
            85, 86, 87],
           dtype='int64')

In [71]:
def depth_cleanup(data):
    for col in data.columns:
        if 'hole' in col.lower() and 'id' in col.lower():
            print(f'use {col} as the hole id columns')
            hole=col
            if data[hole].str.contains('XX',na=False).any():
                drop_index=data[data[hole].str.contains('XX',na=False)].index
                data.drop(drop_index,axis=0,inplace=True)
        if 'geo' in col.lower():        
            geo=col
            print(f'use {col} as the Geo column')
        if 'sample' in col.lower() and 'id' in col.lower():        
            sample=col
            print(f'use {col} as the sample_ id column/ drop na samples')
            drop_index=data[data[sample].isna()].index
            data.drop(drop_index,axis=0,inplace=True)
        if data[col].isna().sum()==len(data):
            print(f'dropping {col}')
            data.drop(col,axis=1,inplace=True)
    data.drop(data[(data[hole].isna()==True)& (data[geo].isna()==True)].index,axis=0,inplace=True)
    data.loc[data[data[hole].isna()==True].index,hole]=data.loc[data[data[hole].isna()==True].index,'foldername']
    print ('drop na hole ids')
    na_num=math.floor(data.shape[1]/2)
    data=data.drop(data[data.isna().sum(axis=1)>=(na_num)].index,axis=0)
    print ('drop na rows')
    return data

In [72]:
data=depth_cleanup(data)
data.isna().sum(axis=0)

use Geo as the Geo column
use Sample_ID as the sample_ id column
dropping Work_Order
use Hole_ID as the hole id columns
dropping Description
dropping Unnamed: 7
dropping _2
dropping CHANGE FOR NEW HOLE !!
dropping _3
dropping depth
dropping _4
dropping _5
dropping _6
dropping _7
dropping _8
dropping _9
dropping _10
dropping _11
dropping _12
dropping _13
dropping _14
dropping _15
dropping _16
dropping _17
dropping _18
dropping _19
drop na hole ids
drop na rows


Geo           0
Sample_ID     0
Hole_ID       0
Depth_ft      0
Depth_m       0
foldername    0
filename      0
dtype: int64

In [73]:
data

,Geo,Sample_ID,Hole_ID,Depth_ft,Depth_m,foldername,filename
88,JR,G452077,KM-22-77,700.0,213.36,KM-22-77,KM-22-77 HYP-PKG Samples.xlsx
89,JR,G452078,KM-22-77,710.0,216.41,KM-22-77,KM-22-77 HYP-PKG Samples.xlsx
90,JR,G452079,KM-22-77,720.0,219.46,KM-22-77,KM-22-77 HYP-PKG Samples.xlsx
91,JR,G452080,KM-22-77,730.0,222.50,KM-22-77,KM-22-77 HYP-PKG Samples.xlsx
92,JR,G452081,KM-22-77,740.0,225.55,KM-22-77,KM-22-77 HYP-PKG Samples.xlsx
...,...,...,...,...,...,...,...
1163,RH,C515420,KM-21-30,1720.0,524.26,KM-22-30,KM-21-30 HYP-PKG Samples.xlsx
1164,RH,C515421,KM-21-30,1730.0,527.30,KM-22-30,KM-21-30 HYP-PKG Samples.xlsx
1165,RH,C515422,KM-21-30,1740.0,530.35,KM-22-30,KM-21-30 HYP-PKG Samples.xlsx
1166,RH,C515423,KM-21-30,1750.0,533.40,KM-22-30,KM-21-30 HYP-PKG Samples.xlsx
